In [1]:
import mne
import numpy as np
import pandas as pd
import json

In [2]:
path = f'data_brainvision/'
name_file = 'Metzler0004'
file = path + name_file + '.vhdr'

raw_data = mne.io.read_raw_brainvision(file, preload=True)

Extracting parameters from data_brainvision/Metzler0004.vhdr...
Setting channel info structure...
Reading 0 ... 1823139  =      0.000 ...  1823.139 secs...


In [3]:
events_stimulus, event_stimulus_dict = mne.events_from_annotations(raw_data)

#carregar o dicionário em json
with open('datasets/event_stimulus_new_dict.json') as json_file:
    event_stimulus_new_dict = json.load(json_file)

Used Annotations descriptions: ['Comment/no USB Connection to actiCAP', 'New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S  7', 'Stimulus/S  8', 'Stimulus/S  9', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14', 'Stimulus/S 15', 'Stimulus/S 16', 'Stimulus/S 17', 'Stimulus/S 18', 'Stimulus/S 19', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 22', 'Stimulus/S 23', 'Stimulus/S 24', 'Stimulus/S 25', 'Stimulus/S 26', 'Stimulus/S 27', 'Stimulus/S 28', 'Stimulus/S 29', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 32', 'Stimulus/S 33', 'Stimulus/S 34', 'Stimulus/S 35', 'Stimulus/S 36', 'Stimulus/S 37', 'Stimulus/S 38', 'Stimulus/S 39', 'Stimulus/S 40', 'Stimulus/S 41', 'Stimulus/S 42', 'Stimulus/S 43', 'Stimulus/S 44', 'Stimulus/S 45', 'Stimulus/S 46', 'Stimulus/S 47', 'Stimulus/S 48', 'Stimulus/S 49', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 52', 'Stimulus/S 53', 'Stimulus/S

In [4]:
#Calcular a amplitude de cada evento em relação ao evento anterior
diff_events = np.diff(events_stimulus[:, 0])
diff_events = np.insert(diff_events, 1, 0, axis=0) 

In [5]:
#Transformar em um dataframe
df_events = pd.DataFrame(events_stimulus, columns=['time(ms)', 'channels','stimulus_key'])

stimulus_desc = []
for x in df_events['stimulus_key']:
    for item in event_stimulus_dict.items():
        if item[1] == x:
            stimulus_desc.append(item[0])

stimulus_new_desc = []
for x in df_events['stimulus_key']:
    for item in event_stimulus_new_dict.items():
        if item[1] == x:
            stimulus_new_desc.append(item[0])
            
df_events['stimulus_desc'] = stimulus_desc
df_events['stimulus_new_desc'] = stimulus_new_desc
df_events['diff(ms)'] = diff_events
df_events

,time(ms),channels,stimulus_key,stimulus_desc,stimulus_new_desc,diff(ms)
0,0,0,99999,New Segment/,New Segment,0
1,0,0,10001,Comment/no USB Connection to actiCAP,USB Connection,0
2,29172,0,157,Stimulus/S157,Stimulus/48_0_R,29172
3,29173,0,153,Stimulus/S153,Stimulus/48_0,1
4,31201,0,251,Stimulus/S251,Answer/Equals,2028
...,...,...,...,...,...,...
310,1789496,0,251,Stimulus/S251,Answer/Equals,1766
311,1796613,0,47,Stimulus/S 47,Stimulus/6_100_R,7117
312,1800729,0,252,Stimulus/S252,Answer/Different,4116
313,1807840,0,57,Stimulus/S 57,Stimulus/8_0,7111


In [6]:
#Salvar como csv
df_events.to_csv(f'datasets/{name_file}_events_stimulus.csv', index=False)